# 引入所需模块

In [0]:
import re
import json
import requests
import numpy as np
import scipy.integrate as spi
import matplotlib.pyplot as plt

## 获取丁香园数据计算治愈率、死亡率

In [0]:
url = 'https://3g.dxy.cn/newh5/view/pneumonia'

response = requests.get(url)
origin = json.loads(re.search(
    r'window.getStatisticsService = ({.*?})', response.text, re.DOTALL).group(1))

data['确诊'] = origin['confirmedCount']
data['疑似'] = origin['suspectedCount']
data['死亡'] = origin['deadCount']
data['治愈'] = origin['curedCount']

data

cure_rate = data['治愈'] / data['确诊']
death_rate = data['死亡'] / data['确诊']

## SI-Model

> SI 模型非常简化，它说健康人一旦感染，就将终生具有传染性，典型的就是 HIV 或者是生化危机这样。

S: Susceptibles，易感者，可能被感染的健康人

I: The Infected，感染者，即患者

β: 病毒传染给健康者的概率

γ: 疾病治愈率（在 SI 模型中，疾病治愈为 0）

INI: 初始状态下易感者与感染者的比例



In [0]:
class SI(object):
    def __init__(self, beta=0.2586, T=150):
        self.beta = beta
        self.gamma = 0
        self.I = 1e-6
        self.S = 1 - self.I
        self.T = T
        self.INI = (self.S, self.I)
        self.T_range = np.arange(0, self.T + 1)
    
    def calculate(self, prop, _):
        Y = np.zeros(2)
        X = prop
        Y[0] = - self.beta * X[0] * X[1] + self.gamma * X[1] # 易感者
        Y[1] = self.beta * X[0] * X[1] - self.gamma * X[1] # 感染者
        return Y
    
    def plot(self):
        RES = spi.odeint(self.calculate, self.INI, self.T_range)
        plt.plot(RES[:,0],color = 'darkblue',label = 'Susceptibles',marker = '.')
        plt.plot(RES[:,1],color = 'red',label = 'The Infected',marker = '.')
        plt.title('SI Model')
        plt.legend()
        plt.xlabel('Day')
        plt.ylabel('Proportion')
        plt.show()

In [0]:
SI_ = SI()
SI_.plot()

## SIS-Model

> 有一些小病，感染者自己会恢复健康或者治好，典型的就是小感冒，人们会在 S 和 I 之间反复横跳。

S: Susceptibles，易感者，可能被感染的健康人

I: The Infected，感染者，即患者

β: 病毒传染给健康者的概率

γ: 疾病治愈率

INI: 初始状态下易感者与感染者的比例



In [0]:
class SIS(object):
    def __init__(self, beta=0.2586, gamma=0.018,T=150):
        self.beta = beta
        self.gamma = gamma
        self.I = 1e-6
        self.S = 1 - self.I
        self.T = T
        self.INI = (self.S, self.I)
        self.T_range = np.arange(0, self.T + 1)
    
    def calculate(self, prop, _):
        Y = np.zeros(2)
        X = prop
        Y[0] = - self.beta * X[0] * X[1] + self.gamma * X[1] # 易感者
        Y[1] = self.beta * X[0] * X[1] - self.gamma * X[1] # 感染者
        return Y
    
    def plot(self):
        RES = spi.odeint(self.calculate, self.INI, self.T_range)
        plt.plot(RES[:,0],color = 'darkblue',label = 'Susceptible',marker = '.')
        plt.plot(RES[:,1],color = 'red',label = 'The Infected',marker = '.')
        plt.title('SIS Model')
        plt.legend()
        plt.xlabel('Day')
        plt.ylabel('Proportion')
        plt.show()

In [0]:
SIS_ = SIS(gamma=cure_rate)
SIS_.plot()

## SIR-Model

> 易感者会不断变为感染者，而感染者又会不断治愈变成康复者，并且不再被感染

S: Susceptibles，易感者，可能被感染的健康人

I: The Infected，感染者，即患者

R: The Recovered，康复者

β: 病毒传染给健康者的概率

γ: 疾病治愈率

INI: 初始状态下易感者、感染者与治愈者的比例


In [0]:
class SIR(object):
    def __init__(self, beta=0.2586, gamma=0.018, T=150):
        self.beta = beta
        self.gamma = gamma
        self.I = 1e-6
        self.R = 1e-6 / 2
        self.S = 1 - self.I - self.R
        self.T = T
        self.INI = (self.S, self.I, self.R)
        self.T_range = np.arange(0, self.T + 1)
    
    def calculate(self, prop, _):
        Y = np.zeros(3)
        X = prop
        Y[0] = - self.beta * X[0] * X[1] # 易感者
        Y[1] = self.beta * X[0] * X[1] - self.gamma * X[1] # 感染者
        Y[2] = self.gamma * X[1] # 治愈者
        return Y
    
    def plot(self):
        RES = spi.odeint(self.calculate, self.INI, self.T_range)
        plt.plot(RES[:,0],color = 'darkblue',label = 'Susceptibles',marker = '.')
        plt.plot(RES[:,1],color = 'red',label = 'The Infected',marker = '.')
        plt.plot(RES[:,2],color = 'green',label = 'The Recovered',marker = '.')
        plt.title('SIR Model')
        plt.legend()
        plt.xlabel('Day')
        plt.ylabel('Proportion')
        plt.show()

In [0]:
SIR_ = SIR(gamma=cure_rate)
SIR_.plot()

## SEIR-Model

> 复杂的病毒，健康的人感染后会先经历潜伏期，一段时间之后才会出现症状。

S: Susceptibles，易感者，可能被感染的健康人

E: The Exposed，潜伏者

I: The Infected，感染者，即患者

R: The Recovered，康复者

β: 病毒传染给健康者的概率

γ1: 潜伏期治愈率（假设为 0）

γ2: 感染者治愈率

α: 潜伏者转换为感染者的比例（假设为 1）

INI: 初始状态下易感者、潜伏者、感染者、治愈者的比例


In [0]:
class SEIR(object):
    def __init__(self, beta=0.2586, gamma_2=0.018, T=150):
        self.beta = beta
        self.gamma_1 = 0
        self.gamma_2 = gamma_2
        self.alpha = 1
        self.I = 1e-6
        self.E = self.I * 41
        self.R = 1e-6 / 2
        self.S = 1 - self.I - self.E - self.R
        self.T = T
        self.INI = (self.S, self.E, self.I, self.R)
        self.T_range = np.arange(0, self.T + 1)
    
    def calculate(self, prop, _):
        Y = np.zeros(4)
        X = prop
        Y[0] = - self.beta * X[0] * X[1] # 易感者
        Y[1] = self.alpha * X[2] - self.gamma_2 * X[1] # 感染者
        Y[2] = self.beta * X[0] * X[1] - (self.alpha + self.gamma_1) * X[2] # 潜伏者
        Y[3] = self.gamma_1 * X[2] + self.gamma_2 * X[1] # 治愈者
        return Y
    
    def plot(self):
        RES = spi.odeint(self.calculate, self.INI, self.T_range)
        plt.plot(RES[:,0],color = 'darkblue',label = 'Susceptibles',marker = '.')
        plt.plot(RES[:,1],color = 'red',label = 'The Infected',marker = '.')
        plt.plot(RES[:,2],color = 'orange',label = 'The Exposed',marker = '.')
        plt.plot(RES[:,3],color = 'green',label = 'The Recovered',marker = '.')
        plt.title('SEIR Model')
        plt.legend()
        plt.xlabel('Day')
        plt.ylabel('Proportion')
        plt.show()

In [0]:
SEIR_ = SEIR(gamma_2=cure_rate)
SEIR_.plot()

## SEIR-Model（考虑死亡率）

> 复杂的病毒，健康的人感染后会先经历潜伏期，一段时间之后才会出现症状。

S: Susceptibles，易感者，可能被感染的健康人

E: The Exposed，潜伏者

I: The Infected，感染者，即患者

R: The Recovered，康复者

D: The Dead，死亡者

β: 病毒传染给健康者的概率

γ1: 潜伏期治愈率（假设为 0）

γ2: 感染者治愈率

α: 潜伏者转换为感染者的比例（假设为 1）

INI: 初始状态下易感者、潜伏者、感染者、治愈者、死亡者的比例

θ: 死亡率


In [0]:
class SEIRD(object):
    def __init__(self, beta=0.2586, gamma_2=0.018, theta=0.022, T=150):
        self.beta = beta
        self.gamma_1 = 0
        self.gamma_2 = gamma_2
        self.alpha = 1
        self.theta = theta
        self.I = 1e-6
        self.E = self.I * 41
        self.R = 1e-6 / 2
        self.S = 1 - self.I - self.E - self.R
        self.D = 0
        self.T = T
        self.INI = (self.S, self.E, self.I, self.R, self.D)
        self.T_range = np.arange(0, self.T + 1)
    
    def calculate(self, prop, _):
        Y = np.zeros(5)
        X = prop
        Y[0] = - self.beta * X[0] * X[1] # 易感者
        Y[1] = self.alpha * X[2] - self.gamma_2 * X[1] - self.theta * X[1] # 感染者
        Y[2] = self.beta * X[0] * X[1] - (self.alpha + self.gamma_1) * X[2] # 潜伏者
        Y[3] = self.gamma_1 * X[2] + self.gamma_2 * X[1] # 治愈者
        Y[4] = self.theta * X[1] # 死亡者
        return Y
    
    def plot(self):
        RES = spi.odeint(self.calculate, self.INI, self.T_range)
        plt.plot(RES[:,0],color = 'darkblue',label = 'Susceptibles',marker = '.')
        plt.plot(RES[:,1],color = 'red',label = 'The Infected',marker = '.')
        plt.plot(RES[:,2],color = 'orange',label = 'The Exposed',marker = '.')
        plt.plot(RES[:,3],color = 'green',label = 'The Recovered',marker = '.')
        plt.plot(RES[:,4],color = 'black',label = 'The Dead',marker = '.')
        plt.title('SEIRD')
        plt.legend()
        plt.xlabel('Day')
        plt.ylabel('Proportion')
        plt.show()

In [0]:
SEIRD_ = SEIRD(gamma_2=cure_rate, theta=death_rate)
SEIRD_.plot()